In [1]:
import numpy as np
import cv2
import os

In [2]:
def get_letters_list2(im_name):
    image = cv2.imread(im_name,0)
    image = cv2.bitwise_not(image)
    nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(image, connectivity=4)
    sizes = stats[1:, -1]
    nb_components = nb_components - 1
    min_size = 100
    for i in range(0, nb_components):
        if sizes[i] <= min_size:
            image[output == i + 1] = 0
    edge = 16
    inv_cropped = image[edge:image.shape[0]-edge,edge:image.shape[1]-edge]
    new_image, letters_only_contour, hierarchy = cv2.findContours(inv_cropped,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    letters_list = []
    cropped = cv2.bitwise_not(inv_cropped)
    # sorts contours by x position
    starting_points = []
    for cnt in letters_only_contour:
        x,y,w,h = cv2.boundingRect(cnt)
        starting_points.append(x)
    sorted_letters_only_contour = [x for _,x in sorted(zip(starting_points,letters_only_contour))]
    # append letters to list.  following code detects unually long letters, and assumes the contour found multiple letters
    # splits such contours in half until it's a reasonable length
    for cnt in sorted_letters_only_contour:
        x,y,w,h = cv2.boundingRect(cnt)
        temp_width = w
        while temp_width > 34:
            if temp_width < 60:
                temp_width = np.floor_divide(temp_width,2)
            elif temp_width < 91:
                temp_width = np.floor_divide(temp_width,3)
            else:
                temp_width = np.floor_divide(temp_width,2)
        for i in range(np.floor_divide(w,temp_width)):
            letters_list.append(cropped[y:y+h,x+i*temp_width:x+(i+1)*temp_width])
    while len(letters_list)<6:
        letter_widths = []
        for letter in letters_list:
            letter_widths.append(letter.shape[1])
        widest_letter_index = letter_widths.index(max(letter_widths))
        widest_letter = letters_list[widest_letter_index]
        letters_list.remove(widest_letter)
        letters_list.insert(widest_letter_index,widest_letter[:,np.floor_divide(widest_letter.shape[1],2):])
        letters_list.insert(widest_letter_index,widest_letter[:,0:np.floor_divide(widest_letter.shape[1],2)])
    if len(letters_list)>6:
        return ('An error has occurred.  Too many letters detected',letters_list)
    else:
        return letters_list

# View samples

In [3]:
image_list = os.listdir('grainy_samples/_10samples')
for image_name in image_list:
    letters_list = get_letters_list2('grainy_samples/_10samples/%s'%image_name)
    for letter in letters_list:
        cv2.imshow('letters only',letter)
        cv2.waitKey(0)
        cv2.destroyAllWindows()  

# Save samples

In [6]:
image_list = os.listdir('grainy_samples/samples')
num_images = len(image_list)
count = 0
errors = 0
for image_name in image_list:
    letters_list = get_letters_list2('grainy_samples/samples/%s'%image_name)
    if len(letters_list)==6:
        for i in range(len(letters_list)):
            letter_key = image_name[i]
            cv2.imwrite('grainy_samples/letters/%s/%s.png'%(letter_key,count),letters_list[i])
            count+=1
    else:
        errors+=1
    image_index = image_list.index(image_name)
    if image_index % 200 ==0:
        print('%d images processed of %d'%(image_index,num_images))
print('%d files could not not be parsed (errors)'%errors)

0 images processed of 16020


C:\Users\Michael\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


200 images processed of 16020
400 images processed of 16020
600 images processed of 16020
800 images processed of 16020
1000 images processed of 16020
1200 images processed of 16020
1400 images processed of 16020
1600 images processed of 16020
1800 images processed of 16020
2000 images processed of 16020
2200 images processed of 16020
2400 images processed of 16020
2600 images processed of 16020
2800 images processed of 16020
3000 images processed of 16020
3200 images processed of 16020
3400 images processed of 16020
3600 images processed of 16020
3800 images processed of 16020
4000 images processed of 16020
4200 images processed of 16020
4400 images processed of 16020
4600 images processed of 16020
4800 images processed of 16020
5000 images processed of 16020
5200 images processed of 16020
5400 images processed of 16020
5600 images processed of 16020
5800 images processed of 16020
6000 images processed of 16020
6200 images processed of 16020
6400 images processed of 16020
6600 images 

In [5]:
image_list[0]

'aaavcc.jpg'